In [410]:
import pandas as pd
import json

In [411]:
fec_data = pd.read_csv('data/FEC_data.csv')
FEC_subset_columns = ['year', 'CAND_ID', 'CAND_NAME', 'CAND_ICI', 'CAND_PTY_AFFILIATION', 'TTL_DISB', 'CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT']

fec_data = fec_data[FEC_subset_columns]
fec_data.rename(columns={'CAND_ID': 'FEC_candidate_id'}, inplace=True)


In [412]:
house_data = pd.read_csv('data/1976-2022-house.csv')
drop_columns = ['state_cen', 'state_ic', 'version', 'fusion_ticket', 'writein', 'unofficial', 'mode', 'stage', 'state_fips', 'office']

house_data = house_data[(house_data['writein'] == False) & (house_data['unofficial'] == False) & (house_data['mode'] == 'TOTAL') & (house_data['stage'] == 'GEN')]

with pd.option_context('mode.chained_assignment', None):
    house_data.rename(columns={'candidate': 'candidate_name', 'candidatevotes': 'candidate_votes', 'totalvotes': 'total_votes'}, inplace=True)
    house_data.drop(columns=drop_columns, inplace=True)

# merging

In [413]:
from fuzzywuzzy import fuzz

# Function to find the best match between two name lists
def find_best_match(name, candidate_list):
    best_match = None
    best_ratio = 0
    for candidate in candidate_list:
        ratio = fuzz.token_sort_ratio(name, candidate)
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = candidate
    return best_match if best_ratio > 60 else None  # Return None if no good match found

In [414]:
pd.options.mode.chained_assignment = None
# Initialize an empty list to store all merged data
all_merged_data = []

for year in range(2010, 2022, 2):
    house_data_year = house_data[house_data['year'] == year]
    fec_data_year = fec_data[fec_data['year'] == year]

    # Ensure district is of the same type in both dataframes
    fec_data_year['CAND_OFFICE_DISTRICT'] = fec_data_year['CAND_OFFICE_DISTRICT'].astype(str)
    house_data_year['district'] = house_data_year['district'].astype(str)

    # Group finance data by state and district
    finance_grouped = fec_data_year.groupby(['CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT'])

    # Iterate through each district in house_data
    for (state, district), house_group in house_data_year.groupby(['state_po', 'district']):
        finance_group = finance_grouped.get_group((state, district)) if (state, district) in finance_grouped.groups else None
        
        if finance_group is not None:
            house_candidates = house_group['candidate_name'].tolist()
            finance_candidates = finance_group['CAND_NAME'].tolist()
            
            for _, house_row in house_group.iterrows():
                best_match = find_best_match(house_row['candidate_name'], finance_candidates)
                if best_match:
                    finance_row = finance_group[finance_group['CAND_NAME'] == best_match].iloc[0]
                    merged_row = house_row.to_dict()
                    merged_row.update(finance_row.to_dict())
                    all_merged_data.append(merged_row)
                # If there's no match, append the row with only house data
                else:
                    merged_row = house_row.to_dict()
                    all_merged_data.append(merged_row)

# Create the final merged dataframe with all years
merged_df = pd.DataFrame(all_merged_data)
merged_df.drop(columns=['CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT'], inplace=True)

merged_df['district_id'] = merged_df['state_po'] + "_" + merged_df['district']
merged_df = merged_df[['district_id'] + [col for col in merged_df.columns if col != 'district_id']]

merged_df['race_id'] = merged_df['state_po'] + "_" + merged_df['district'] + "_" + merged_df['year'].astype(str)
merged_df = merged_df[['race_id'] + [col for col in merged_df.columns if col != 'race_id']]

# Save the merged dataset with all years to a single CSV file
merged_df = merged_df[merged_df['state'] != "DISTRICT OF COLUMBIA"]

# cleaning up parties we are not interested in
merged_df = merged_df.dropna(subset=['party'])
merged_df['party'] = merged_df['party'].apply(lambda x: 'Democrat' if x.lower() == 'democrat' else 'Republican' if x.lower() == 'republican' else 'Other')

# candidate id
merged_df['candidate_id'] = merged_df['year'].astype(str) + "_" + merged_df['FEC_candidate_id']

In [415]:
merged_df[merged_df['candidate_id']=="2018_H2NY03089"]

,race_id,district_id,year,state,state_po,district,runoff,special,candidate_name,party,candidate_votes,total_votes,FEC_candidate_id,CAND_ICI,TTL_DISB,candidate_id
5918,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Other,12504,241217,H2NY03089,I,3183657.13,2018_H2NY03089
5919,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Other,2535,241217,H2NY03089,I,3183657.13,2018_H2NY03089
5920,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Other,474,241217,H2NY03089,I,3183657.13,2018_H2NY03089
5921,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Republican,112565,241217,H2NY03089,I,3183657.13,2018_H2NY03089
5922,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Other,0,241217,H2NY03089,I,3183657.13,2018_H2NY03089


In [416]:
# Group by candidate_id and aggregate
grouped_df = merged_df.groupby('candidate_id').agg({
    'race_id': 'first',
    'district_id': 'first',
    'year': 'first',
    'state': 'first',
    'state_po': 'first',
    'district': 'first',
    'runoff': 'first',
    'special': 'first',
    'candidate_name': 'first',
    'party': lambda x: 'Democrat' if 'Democrat' in x.values else ('Republican' if 'Republican' in x.values else 'Other'),
    'candidate_votes': 'sum',
    'total_votes': 'first',
    'FEC_candidate_id': 'first',
    'CAND_ICI': 'first',
    'TTL_DISB': 'first'
}).reset_index()

grouped_df = grouped_df[(grouped_df['runoff'] != True)]
grouped_df = grouped_df[grouped_df['special'] != True]

grouped_df.drop(columns=['state_po', 'district', 'runoff', 'special'], inplace=True)


In [417]:
grouped_df.sort_values(by=['race_id'], inplace=True)
grouped_df.to_csv('data/cleaned/merged_house_finance_data_all_years.csv', index=False)

# switching to district as row

In [418]:
grouped_df

,candidate_id,race_id,district_id,year,state,candidate_name,party,candidate_votes,total_votes,FEC_candidate_id,CAND_ICI,TTL_DISB
0,2010_H0AK00089,AK_0_2010,AK_0,2010,ALASKA,HARRY T CRAWFORD JR,Democrat,77606,254335,H0AK00089,C,235571.43
636,2010_H6AK00045,AK_0_2010,AK_0,2010,ALASKA,DON YOUNG,Republican,175384,254335,H6AK00045,I,887310.33
1081,2012_H2AK00119,AK_0_2012,AK_0,2012,ALASKA,SHARON M CISSNA,Democrat,82927,289804,H2AK00119,C,24388.00
1569,2012_H6AK00045,AK_0_2012,AK_0,2012,ALASKA,DON YOUNG,Republican,185296,289804,H6AK00045,I,665974.39
2406,2014_H6AK00045,AK_0_2014,AK_0,2014,ALASKA,DON YOUNG,Republican,142572,279741,H6AK00045,I,809039.20
...,...,...,...,...,...,...,...,...,...,...,...,...
3665,2018_H6WY00159,WY_0_2018,WY_0,2018,WYOMING,LIZ CHENEY,Republican,127963,201245,H6WY00159,I,715739.91
3472,2018_H2WY00133,WY_0_2018,WY_0,2018,WYOMING,DANIEL CLYDE CUMMINGS,Other,6070,201245,H2WY00133,C,485.00
4128,2018_H8WY01062,WY_0_2018,WY_0,2018,WYOMING,GREG HUNTER,Democrat,59903,201245,H8WY01062,C,47069.51
4551,2020_H0WY01036,WY_0_2020,WY_0,2020,WYOMING,LYNNETTE GREY BULL,Democrat,66576,278503,H0WY01036,C,132285.65


In [201]:
import json

# Load the JSON data from the file
with open('data/wiki/wiki_scrape_nhe.json', 'r') as file:
    nhe_data = json.load(file)

with open('data/wiki/wiki_scrape_pvi.json', 'r') as file:
    pvi_data = json.load(file)

In [202]:
for year, data in nhe_data.items():
    republican_percentage = 0
    democratic_percentage = 0
    
    for party_data in data:
        if party_data['party'] == 'Republican Party':
            republican_percentage = party_data['percentage']
        elif party_data['party'] == 'Democratic Party':
            democratic_percentage = party_data['percentage']
    percent_difference = republican_percentage - democratic_percentage
    
    # Update the 'NHE' column for the corresponding year
    merged_df.loc[merged_df['year'] == int(year), 'NHE'] = round(percent_difference, 2)
    
    # Print the year and the difference
    print(f"Year: {year}, Percent difference between Republican and Democratic votes: {percent_difference:.2f}%")

Year: 2022, Percent difference between Republican and Democratic votes: 2.72%
Year: 2020, Percent difference between Republican and Democratic votes: -3.10%
Year: 2018, Percent difference between Republican and Democratic votes: -8.60%
Year: 2016, Percent difference between Republican and Democratic votes: 1.10%
Year: 2014, Percent difference between Republican and Democratic votes: 5.70%
Year: 2012, Percent difference between Republican and Democratic votes: -1.10%
Year: 2010, Percent difference between Republican and Democratic votes: 6.80%
Year: 2008, Percent difference between Republican and Democratic votes: -10.60%


In [203]:
def convert_pvi(pvi):
    if 'R+' in pvi:
        return int(pvi.replace('R+', ''))
    elif 'D+' in pvi:
        return -int(pvi.replace('D+', ''))
    elif 'even' in pvi.lower():
        return 0
    else:
        print(f"PVI value not recognized: {pvi}")
        return None


In [208]:
for year, state_data in pvi_data.items():
    for state, districts in state_data.items():
        for district_info in districts:
            district_name = district_info['district']
            pvi = district_info['pvi']
            
            # Extract district number
            if 'at-large' in district_name:
                district_number = 0
            elif " " in district_name:
                district_number = district_name.split(' ')[-1]
                if "\xa0" in district_name:
                    district_number = int(district_name.split('\xa0')[-1])
                else:
                    district_number = int(district_name.split(' ')[-1])
            else:
                district_number = int(district_name.split('\u00a0')[-1])
            
            # Update merged_df with PVI information
            mask = (
                (merged_df['year'] == int(year)) &
                (merged_df['state'] == state.upper()) &
                (merged_df['district'] == str(district_number))
            )
            converted_pvi = convert_pvi(pvi)
            merged_df.loc[mask, 'pvi'] = converted_pvi


In [210]:
merged_df = merged_df[merged_df['state'] != "DISTRICT OF COLUMBIA"]
merged_df.to_csv('data/cleaned/merged_house_finance_data_all_years.csv', index=False)